
HuggingFace Transformers를 활용한 토큰 분류 모델 학습

본 노트북에서는 `klue/roberta-base` 모델을 **KLUE** 내 **NLI** 데이터셋을 활용하여 모델을 훈련하는 예제를 다루게 됩니다.


학습 과정 이후에는 간단한 예제 코드를 통해 모델이 어떻게 활용되는지도 함께 알아보도록 할 것입니다.

모든 소스 코드는 [`huggingface-tutorial`](https://huggingface.co/course/chapter7/2)를 참고하였습니다. 

먼저, 노트북을 실행하는데 필요한 라이브러리를 설치합니다. 모델 훈련을 위해서는 `transformers`가, 학습 데이터셋 로드를 위해서는 `datasets` 라이브러리의 설치가 필요합니다. 그 외 모델 성능 검증을 위해 `scipy`, `scikit-learn`을 추가로 설치해주도록 합니다.

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import evaluate
evaluate.__version__

'0.4.0'

In [2]:
!pip install  evaluate 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.9 MB/s eta 0:00:00


In [7]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=cc649d41840f2a9314ea044ba2598a45a7550518281b39ff38280b9b44c1df48
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [1]:
!pip install -U transformers datasets scipy scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.3 MB/s eta 0:00:00


In [ ]:
#from huggingface_hub import notebook_login

#notebook_login()




# MRC

노트북을 실행하는데 필요한 라이브러리들을 모두 임포트합니다.

In [ ]:
import random
import logging
from IPython.display import display, HTML
import torch

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer

학습에 필요한 정보를 변수로 기록합니다.

본 노트북에서는 `klue-roberta-base` 모델을 활용하지만, https://huggingface.co/klue 페이지에서 더 다양한 사전학습 언어 모델을 확인하실 수 있습니다.

학습 태스크로는 `nli`를, 배치 사이즈로는 32를 지정하겠습니다.

In [ ]:
model_checkpoint = "bespin-global/klue-bert-base-mrc"
batch_size = 2
task = "mrc"

이제 HuggingFace `datasets` 라이브러리에 등록된 KLUE 데이터셋 중, NLI 데이터를 내려받습니다.

In [ ]:
#['ynat', 'sts', 'nli', 'ner', 're', 'dp', 'mrc', 'wos']
raw_datasets = load_dataset("klue", task)

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

다운로드 혹은 로드 후 얻어진 `datasets` 객체를 살펴보면, 훈련 데이터와 검증 데이터가 포함되어 있는 것을 확인할 수 있습니다.

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []
    
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,title,context,news_category,source,guid,is_impossible,question_type,question,answers
0,유가족 “세상에서 가장 슬픈 신분증 차고 와”,"16일 오후 청와대에서 이뤄진 박근혜 대통령과 세월호 참사 유가족 대표들 간 대화는 1시간20분간 진행됐다. 이날 면담은 박 대통령의 제안으로 이뤄졌다. 전날 저녁 유가족 측에 면담의사가 전달됐고 유가족들의 내부 논의를 거쳐 17명이 면담자로 선정됐다.박 대통령은 유가족들과 대화 도중 두 번의 사과 발언을 했다. 대화 끝 부분에서는 한 유가족이 “대통령을 만나려고 진도교회까지 걸어갔는데 그게 귀와 눈까지 들어갔는지…. 그런 부분에서 분노와 허무감을 느꼈다”고 말하자 “자꾸 위로해 드리고 슬픔 속에서도 견딜 수 있도록 해드려야 하는데 여러 가지로 부족한 점이 많았다. 그런 부분에 가슴 아프게 생각하며 다시 한번 사과를 드린다”고 했다. 박 대통령이 유족들 앞에서 ‘대면사과’를 한 것은 처음이다. 가슴 아픈 사연을 듣는 과정에서는 한 차례 눈물을 흘리기도 했다.유가족들은 사고 현장을 지키며 느꼈던 답답함과 비통함 등을 토로했다. 한 유족은 “오늘 청와대에 들어오면서 세상에서 가장 슬픈 신분증을 차고 왔다”며 “근데 저는 제 아이를 포함한 많은 희생자들이 우리나라 역사, 또 세계 역사에 남을 수 있도록 가장 가치있고 고귀하게 만들어주시는 것이 대통령을 포함한 정부의 책임이라고 생각한다”고 말했다.다른 유족은 “총리라는 분한테 ‘어떤 방안을 갖고 있느냐’고 물었더니 모 장관한테 ‘전화걸어봐’라고 했다. 또 그 장관도 모르니 기조실장한테 전화를 걸었다. 애들은 물속에 있는데, 국무총리나 장관이 그런 얘기를 할 때 얼마나 가슴이 찢어졌는지 모른다”고 털어놨다. 한 유족은 “큰애를 잃고 둘째가 극도의 심리적 불안을 느껴 자살충동이 심하다는 진단을 받았다. 제2의 피해가 발생하지 않도록 대책을 마련해달라”고 요구했다.유가족들은 세월호 사건을 정치적으로 이용하려는 움직임에 대해 거부감도 드러냈다. 한 유족은 “여야가 세월호 사건을 자꾸 정치적으로 이용하려 하는데, 그런 것 좀 안 했으면 좋겠다”며 “여야 정치인들한테 꼭 당부해달라”고 말했다. 대화 끝 부분에 다른 유족은 “아직 저희 속이 확 뚫리지는 않아서 조금 아쉽다”고 말했고, 박 대통령은 “부족한 점이 있으면 또 말씀해달라”고 했다.",종합,hankyung,klue-mrc-v1_train_07932,True,3,면담자로 선정된 정치인의 수는?,"{'answer_start': [126, 126], 'text': ['17명', '17']}"
1,제임스 맥헨리,"영국과 북미 식민지 간의 관계가 악화되어 1775년 4월에 뉴잉글랜드에서 렉싱턴 콩코드 전투가 시작되자 맥헨리는 반 영국 입장을 표명했다. 맥헨리는 보스턴 공성전에서 대륙군에 자원하여 참여하고 매사추세츠 만 식민지 케임브리지 군 병원에서 군의관 보조로 일했다.\n\n1775년 12월, 리처드 몽고메리 장군이 이끄는 대륙군이 캐나다의 퀘벡 전투에서 대패하면서 맥헨리는 부상자의 치료를 위해 설치된 뉴욕 식민지 북부군 병원 중 하나로 가도록 요청받았다. 하지만 맥헨리는 부임하기 직전에 추가로 필요한 의약품을 모아야 한다는 이유로 필라델피아에 돌아왔다.\n\n1776년 필라델피아로 돌아온 맥헨리는 벤자민 러쉬의 추천에 의해 로버트 메가우 대령의 연대에 군의관에 기용되었다. 1776년 8월 10일, 맥헨리는 22살의 나이로 펜실베이니아 제5 대대의 정규 대원으로 다시 필라델피아를 떠나 전선으로 향했다. 펜실베이니아 제5 대대는 대륙군의 정규 부대로 뉴저지 식민지 북부에 배치되어 뉴욕과 직접 연락을 끊었다. 펜실베이니아 제5 대대는 맨해턴 섬 북서쪽 끝에 대륙군 전초 기지로서의 워싱턴 요새를 건설하는 임무를 할당했다. 당초 대륙군은 워싱턴 요새를 거점으로 뉴욕을 방어, 허드슨 강에 영국군의 침입을 막아낼 계획이었다. 따라서 펜실베이니아 제5 연대는 워싱턴 요새 완성 후에도 그 방위를 맡았지만, 1776년 8월에 영국군 사령관 윌리엄 하우에게 급습을 받아 같은 해 11월 16일에 워싱턴 요새를 함락 당했다. 맥헨리는 2000여명의 대륙군 병사와 함께 영국군의 포로가 되었다. 맥헨리는 전쟁 포로의 질병을 관리하고 몇 달을 보낸 후 1777년에 당시 전쟁의 관례에 따라 석방되었다.\n\n1778년 3월, 맥헨리는 다시 대륙군에 참가하여 대륙군 야영지 밸리 포지에서 1개월 정도 군의관을 맡았다. 1778년 5월, 대륙군 총사령관 조지 워싱턴은 맥헨리를 참모부 서기로 임명했다. 맥헨리는 워싱턴과 함께 2년 반 정도를 보냈다. 그동안 맥헨리는 몬머스 전투, 스프링필드 전투, 뉴저지 전투에 참가했고, 헨리 녹스, 알렉산더 해밀턴, 라파예트 등과 함께 워싱턴에서 가장 친한 ‘군 패밀리’의 일원으로 손꼽혔다.\n\n맥헨리는 라파예트와 뜨거운 평생 우정을 쌓았다. 1780년 가을, 맥헨리는 프랑스인 부대에 전속되어 소령으로 임무를 맡았다. 맥헨리는 라피예트의 측근으로 군사 작전에 참여했다. 1780년 겨울, 맥헨리는 경보병 부대와 함께 버지니아에 가서, 1781년 가을 전역할 때까지 라파예트의 강행군을 지원했다",None,wikipedia,klue-mrc-v1_train_00236,True,3,멕헨리가 친 영국 입장을 표명한 해는?,"{'answer_start': [23, 23], 'text': ['1775년', '1775']}"
2,탈출!저성장 3만달러 넘어 4만달러로‘해외 직구’ 年1조 … 소비 유출,"직장인 김효진 씨(30)는 지난해 말 미국 온라인 오픈마켓인 아마존에서 삼성전자의 51인치 디지털 TV를 현지 판매가격 대비 30%가량 싼 690달러(약 73만원)에 구입했다. 관세(제품가격의 8%)와 부가가치세(10%), 배송 대행 수수료(69달러) 등 추가 비용을 포함해 원화로 총 93만원이 들었다. 그러고도 이득을 봤다. 당시 한국 온라인 몰에서 똑같은 제품이 100만원대에 팔리고 있었기 때문이다. 김씨는 “가격이 싼 데다 제품도 워낙 다양해 해외 온라인 몰을 자주 이용하고 있다”고 말했다. ▶관련시리즈 A4, 5면지난해 해외 직접 구매(직구)가 사상 처음으로 1조원을 돌파했다. 9일 관세청에 따르면 지난해 1~11월까지 국내 온라인 직구족(族)의 해외 주문은 총 980만건으로 금액은 9억1100만달러에 달했다. 12월 수치를 반영하면 1조원을 넘어섰을 게 확실시된다. 이는 같은 기간 롯데백화점 잠실점이 올린 매출과 맞먹는 규모다. 2010년 2억7400만달러였던 해외 직구는 매년 20~30%의 가파른 성장세를 거듭하며 폭발적으로 늘어나고 있다. 소득 정체와 가계부채에 짓눌려 가뜩이나 취약한 소비 기반이 해외 직구 열풍에 냉가슴을 앓고 있는 것이다. 올해 내수 활성화를 전면에 앞세운 박근혜 정부는 최근 이 문제의 심각성을 인식, 대책 마련에 들어갔다.",종합,hankyung,klue-mrc-v1_train_11767,True,3,아마존에서 TV를 현지 판매가격보다 비싸게 산 사람은?,"{'answer_start': [0, 0, 0, 4, 4, 4], 'text': ['직장인 김효진 씨(30)', '직장인 김효진 씨', '직장인 김효진', '김효진', '김효진 씨', '김효진 씨(30)']}"
3,히포캠프 (위성),"해왕성의 가장 거대한 위성 트리톤은 행성을 여타 위성들과 반대 방향으로 돌고 있으며 궤도가 해왕성 적도에 대해 기울어져 있다. 가설에 따르면 해왕성의 자체 위성계가 만들어지고 한참 뒤 해왕성이 카이퍼 대에서 트리톤을 포획해 왔다고 한다. 기존 위성들의 궤도는 이 사건 때문에 변화를 일으켜 위성들 중 일부는 내쳐져 날아가고 나머지는 서로 충돌하여 부서졌을 것이다. 적어도 현존하는 해왕성 내부 위성들 중 일부는 트리톤의 궤도가 조석 감속에 의해 원형으로 바뀐 뒤, 충돌 후 남은 잔해들이 강착되어 만들어진 존재로 보인다. \n\n다른 가설에 따르면 히포캠프는 제일 가까우면서 히포캠프보다 훨씬 거대한 위성 프로테우스에서 떨어져 나온 잔해로부터 만들어졌을 것이라 한다. 이 잔해는 프로테우스에서 가장 큰 충돌구 파로스를 만들었던 혜성 충돌로부터 분출

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 17554
    })
    validation: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 5841
    })
})

In [ ]:
raw_datasets['train'][0]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'news_category': '종합',
 'source': 'hankyung',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'question_type': 1,
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': {'answer_start': [478, 478]

In [ ]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.
Question:  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
Answer:  {'answer_start': [478, 478], 'text': ['한 달가량', '한 달']}


In [ ]:
raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/17554 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 6715
})

In [ ]:
print(raw_datasets["validation"][0]["answers"])
print(raw_datasets["validation"][2]["answers"])

{'answer_start': [666, 666], 'text': ['뉴 740Li 25주년 에디션', '뉴 740Li 25주년']}
{'answer_start': [155, 155], 'text': ['2014년', '2014']}


In [ ]:
print(raw_datasets["validation"][2]["context"])
print(raw_datasets["validation"][2]["question"])

가장 일하고 싶은 회사로 미국 컨설팅 업체인 베인&컴퍼니가 선정됐다. 트위터와 링크트인, 페이스북 등 정보기술(IT) 기업에 대한 선호도 높게 나타났다. 미국 경제전문지 포브스는 “채용 사이트인 글라스도어가 미국에서 일하는 50만명의 직장인을 대상으로 설문조사를 벌여 ‘2014년 일하고 싶은 50대 회사’를 선정했다”고 11일(현지시간) 보도했다. 베인&컴퍼니는 2012년에 이어 두 번째로 1위에 올랐다. 하는 일이 재미있고 일이 영향력이 있으며, 함께 일하는 사람들이 똑똑해서 많이 배울 수 있다는 것 등이 직원들이 베인&컴퍼니에 만족하는 주된 이유였다. 서맨사 주판 글라스도어 대변인은 “베인&컴퍼니 직원들은 늘 새로운 것을 배울 수 있고 도전적인 일이 많아 하루도 지루한 날이 없다고 평가했다”며 “이런 경험은 회사를 그만둔 후에도 개인 경력에 도움이 된다”고 말했다. 트위터가 2위를 차지했다. 트위터 직원들은 복잡하고 중요한 일을 매일 하면서 전 세계 수백만명의 사용자들과 소통할 수 있다는 점을 회사에 다니고 싶은 이유로 꼽았다. 트위터에 이어 링크트인과 페이스북이 각각 3위, 5위를 차지하는 등 IT 기업의 선전도 눈에 띄었다. 글라스도어는 “톱 50 가운데 22개가 IT 기업”이라며 “서로 더 좋은 인재를 영입하고, 기존의 인재를 뺏기지 않기 위해 경쟁하는 것이 더 매력적인 기업환경을 만들기 위한 노력으로 이어지고 있다”고 설명했다. 2009년부터 일하고 싶은 회사를 선정해 발표하고 있는 글라스도어는 2012년 11월부터 지난달까지 회사에 대한 만족도, 임금이나 복지 혜택, 일과 가정 사이의 균형, 경영자에 대한 평가 등 18개의 질문으로 구성된 설문조사를 실시했다. 글라스도어는 “구직자들이 어느 회사에서 일할지 결정하는 데 좋은 정보를 주는 자료”라며 “특히 현장에서 일하는 직원의 평가라는 점에서 리스트에 오른 기업들은 자부심을 가져도 될 것”이라고 평가했다. 상위권에 오른 기업들은 공통점을 가지고 있었다. 미국 경제경영전문지 패스트컴퍼니는 일하고 싶은

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer

BertTokenizerFast(name_or_path='bespin-global/klue-bert-base-mrc', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP] 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다. 17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2 ~ 3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다. 장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20 ~ 21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24 ~ 25일이었으며 장마기간은 32일, 강수일수는 17. 2일이었다. 기상청은 올해 장마기간의 평균 강수량이 350 ~ 400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다. [SEP]'

In [ ]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50, # 자를떄 겹치는 부분
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP] 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다. 17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP]가 시작될 전망이다. 17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2 ~ 3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP] 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2 ~ 3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP]보다 2 ~ 3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다. 장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP]윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다. 장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20 ~ 21일 남부지

In [ ]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50, # 두 개의 연속 청크 사이에 겹치는 토큰 수를 설정합니다(여기서는 50개).
    return_overflowing_tokens=True, # example index --> chunk(feature)들은 같은 index를 공유한다. 
    return_offsets_mapping=True, # 토근의 위치값
)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [ ]:
inputs['input_ids'][0][:10]

[2, 1174, 18956, 26797, 2145, 22152, 2128, 2292, 2097, 26797]

In [ ]:
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
inputs["offset_mapping"] #input[starts[i]:ends[i]]

[[(0, 0),
  (0, 1),
  (1, 4),
  (5, 7),
  (7, 8),
  (9, 11),
  (11, 12),
  (12, 13),
  (13, 14),
  (15, 17),
  (17, 18),
  (19, 21),
  (22, 24),
  (24, 25),
  (26, 29),
  (29, 30),
  (31, 33),
  (33, 34),
  (34, 35),
  (0, 0),
  (0, 2),
  (3, 4),
  (4, 6),
  (7, 8),
  (8, 9),
  (10, 12),
  (12, 13),
  (14, 16),
  (17, 20),
  (21, 23),
  (23, 24),
  (25, 27),
  (27, 28),
  (28, 31),
  (31, 32),
  (33, 35),
  (35, 36),
  (36, 37),
  (37, 38),
  (39, 42),
  (42, 43),
  (44, 46),
  (46, 47),
  (47, 48),
  (48, 49),
  (50, 54),
  (55, 56),
  (57, 59),
  (59, 61),
  (61, 62),
  (62, 64),
  (64, 65),
  (66, 68),
  (68, 69),
  (70, 72),
  (73, 76),
  (77, 78),
  (78, 79),
  (80, 83),
  (83, 84),
  (84, 85),
  (86, 88),
  (89, 91),
  (91, 92),
  (92, 93),
  (93, 94),
  (95, 98),
  (98, 99),
  (100, 101),
  (101, 103),
  (104, 107),
  (107, 108),
  (109, 112),
  (112, 114),
  (114, 115),
  (116, 118),
  (118, 120),
  (121, 122),
  (122, 123),
  (123, 124),
  (125, 127),
  (127, 128),
  (128, 129

In [ ]:
inputs = tokenizer(
    raw_datasets["train"][2:6]["question"],
    raw_datasets["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 4 examples gave 77 features.
Here is where each comes from: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3].


In [ ]:
answers = raw_datasets["train"][2:6]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

print(len(start_positions))
start_positions, end_positions


77


([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  85,
  52,
  19,
  0,
  0,
  0,
  0,
  0,
  80,
  51,
  22,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  72,
  40,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  81,
  58,
  35,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  89,
  56,
  23,
  0,
  0,
  0,
  0,
  0,
  81,
  52,
  23,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  78,
  46,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  87,
  64,
  41,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0])

In [ ]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
        
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/17554 [00:00<?, ? examples/s]

(17554, 37586)

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 37586
})

In [ ]:
raw_datasets["train"][0]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'news_category': '종합',
 'source': 'hankyung',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'question_type': 1,
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': {'answer_start': [478, 478]

앞서 정의한 함수를 활용해 훈련 데이터를 살펴보도록 합시다.

이처럼 데이터를 살펴보는 것의 장점으로는 각 라벨에 어떠한 문장들이 해당하는지에 대한 감을 익힐 수 있다는데에 있습니다.


훈련 과정 중 모델의 성능을 파악하기 위한 메트릭을 설정합니다.

`datasets` 라이브러리에는 이미 구현된 메트릭을 사용할 수 있는 `load_metric` 함수가 있습니다.


In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["guid"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
raw_datasets["validation"]["guid"][0]

'klue-mrc-v1_dev_01891'

In [ ]:
raw_datasets["validation"][0]

{'title': 'BMW 코리아, 창립 25주년 기념 ‘BMW 코리아 25주년 에디션’ 한정 출시',
 'context': 'BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.',
 'news

In [ ]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/5841 [00:00<?, ? examples/s]

(5841, 12645)

In [ ]:
validation_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 12645
})

## small data eval set 

In [ ]:
model_checkpoint = "bespin-global/klue-bert-base-mrc"


In [ ]:
small_eval_set = raw_datasets["validation"].select(range(10))
trained_checkpoint = model_checkpoint

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
tokenizer.pad_token_id

0

In [ ]:

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")




In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [ ]:
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
print(batch)

{'input_ids': tensor([[    2, 10881, 25219,  ..., 26903,  2470,     3],
        [    2, 10881, 25219,  ...,     0,     0,     0],
        [    2,  4022,  7046,  ...,  2145, 13256,     3],
        ...,
        [    2,  3763,  2052,  ...,     0,     0,     0],
        [    2,  8418,  8134,  ...,  8979,  2138,     3],
        [    2,  8418,  8134,  ...,     0,     0,     0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}


In [ ]:
eval_set_for_model["input_ids"]

tensor([[    2, 10881, 25219,  ..., 26903,  2470,     3],
        [    2, 10881, 25219,  ...,     0,     0,     0],
        [    2,  4022,  7046,  ...,  2145, 13256,     3],
        ...,
        [    2,  3763,  2052,  ...,     0,     0,     0],
        [    2,  8418,  8134,  ...,  8979,  2138,     3],
        [    2,  8418,  8134,  ...,     0,     0,     0]])

In [ ]:
batch["input_ids"].shape

torch.Size([22, 384])

In [ ]:
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(device)

In [ ]:
trained_model.resize_token_embeddings(len(tokenizer))


Embedding(32000, 768, padding_idx=0)

  0%|          | 0/10 [00:00<?, ?it/s]

{'exact_match': 70.0, 'f1': 75.0}

In [ ]:
with torch.no_grad():
    outputs = trained_model(**batch)

In [ ]:
outputs


QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-10.4259,  -8.2104, -10.6531,  ...,  -9.2646, -10.1434,  -9.6799],
        [-10.5578,  -7.4443, -10.6112,  ..., -10.4956, -10.4950, -10.5552],
        [-10.8399, -10.3300, -10.4939,  ...,  -9.6453,  -9.5299, -10.0409],
        ...,
        [-10.3440,  -1.4913,  -8.3626,  ..., -10.6030, -10.6283, -10.7861],
        [-10.4383,  -8.3857,  -8.6014,  ...,  -6.6517,  -9.0265,  -9.0437],
        [-10.4451,  -8.3066,  -8.6272,  ..., -10.2783, -10.4552, -10.4121]],
       device='cuda:0'), end_logits=tensor([[-11.1724, -11.5224, -10.8588,  ..., -11.3009, -11.1145, -10.9811],
        [-11.1309, -11.2073, -10.6904,  ..., -11.3716, -11.4081, -11.3470],
        [-11.0662, -11.1134, -10.9050,  ..., -10.9775, -11.0441, -10.6994],
        ...,
        [-10.7839,  -9.0363, -10.3444,  ..., -11.3930, -11.3212, -11.2008],
        [-11.2650, -11.4490, -11.1291,  ...,  -9.2984, -10.4450, -10.3113],
        [-11.2777, -11.3488, -11.1675,  ..., -11

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
start_logits.shape

(22, 384)

In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [ ]:
example_to_features

defaultdict(list,
            {'klue-mrc-v1_dev_01891': [0, 1],
             'klue-mrc-v1_dev_01842': [2, 3],
             'klue-mrc-v1_dev_03566': [4, 5],
             'klue-mrc-v1_dev_02673': [6, 7],
             'klue-mrc-v1_dev_05664': [8, 9],
             'klue-mrc-v1_dev_00170': [10, 11],
             'klue-mrc-v1_dev_04276': [12, 13, 14],
             'klue-mrc-v1_dev_03749': [15, 16, 17],
             'klue-mrc-v1_dev_00114': [18, 19],
             'klue-mrc-v1_dev_04087': [20, 21]})

In [ ]:
import numpy as np

n_best = 20
max_answer_length = 50
predicted_answers = []

for example in small_eval_set:
    example_id = example["guid"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [ ]:
predicted_answers

[{'id': 'klue-mrc-v1_dev_01891',
  'prediction_text': '뉴 840i xDrive 그란쿠페 25주년 에디션'},
 {'id': 'klue-mrc-v1_dev_01842', 'prediction_text': '독일 뒤셀도르프로'},
 {'id': 'klue-mrc-v1_dev_03566', 'prediction_text': '2014'},
 {'id': 'klue-mrc-v1_dev_02673', 'prediction_text': '페이스북'},
 {'id': 'klue-mrc-v1_dev_05664', 'prediction_text': '링크트인'},
 {'id': 'klue-mrc-v1_dev_00170', 'prediction_text': '마드리드'},
 {'id': 'klue-mrc-v1_dev_04276', 'prediction_text': '국제 원자재값 하락'},
 {'id': 'klue-mrc-v1_dev_03749', 'prediction_text': '2조6000억원'},
 {'id': 'klue-mrc-v1_dev_00114', 'prediction_text': '즈루치요'},
 {'id': 'klue-mrc-v1_dev_04087', 'prediction_text': '‘운영허가 인증서'}]

In [ ]:
import evaluate

metric = evaluate.load("squad")

In [ ]:
theoretical_answers = [
    {"id": ex["guid"], "answers": ex["answers"]} for ex in small_eval_set
]

In [ ]:
print(predicted_answers)


[{'id': 'klue-mrc-v1_dev_01891', 'prediction_text': '뉴 840i xDrive 그란쿠페 25주년 에디션'}, {'id': 'klue-mrc-v1_dev_01842', 'prediction_text': '독일 뒤셀도르프로'}, {'id': 'klue-mrc-v1_dev_03566', 'prediction_text': '2014'}, {'id': 'klue-mrc-v1_dev_02673', 'prediction_text': '페이스북'}, {'id': 'klue-mrc-v1_dev_05664', 'prediction_text': '링크트인'}, {'id': 'klue-mrc-v1_dev_00170', 'prediction_text': '마드리드'}, {'id': 'klue-mrc-v1_dev_04276', 'prediction_text': '국제 원자재값 하락'}, {'id': 'klue-mrc-v1_dev_03749', 'prediction_text': '2조6000억원'}, {'id': 'klue-mrc-v1_dev_00114', 'prediction_text': '즈루치요'}, {'id': 'klue-mrc-v1_dev_04087', 'prediction_text': '‘운영허가 인증서'}]


In [ ]:
print(predicted_answers[9])
print(theoretical_answers[9])

{'id': 'klue-mrc-v1_dev_04087', 'prediction_text': '‘운영허가 인증서'}
{'id': 'klue-mrc-v1_dev_04087', 'answers': {'answer_start': [215, 216], 'text': ['‘운영허가 인증서’', '운영허가 인증서']}}


In [ ]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 40.0, 'f1': 56.0}

In [ ]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["guid"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["guid"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
compute_metrics(start_logits, end_logits, eval_set, small_eval_set)

  0%|          | 0/10 [00:00<?, ?it/s]

{'exact_match': 40.0, 'f1': 56.0}

In [ ]:
del trained_model

# Let's Train the model from Scratch

In [ ]:
model_checkpoint = "klue/bert-base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

In [ ]:
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
next(model.parameters()).is_cuda

True

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "squad-test",
    per_device_train_batch_size = 16,
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True
)

In [ ]:
!nvidia-smi

Sun Mar 19 13:48:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    30W /  70W |   1967MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.940700
1000,1.324100
1500,1.206800
2000,1.134800
2500,1.032200
3000,0.791100
3500,0.745000
4000,0.748900
4500,0.728600
5000,0.618700


TrainOutput(global_step=7050, training_loss=0.8721943280375596, metrics={'train_runtime': 3592.2463, 'train_samples_per_second': 31.389, 'train_steps_per_second': 1.963, 'total_flos': 2.2097474772028416e+16, 'train_loss': 0.8721943280375596, 'epoch': 3.0})

In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

  0%|          | 0/5841 [00:00<?, ?it/s]

{'exact_match': 67.2145180619757, 'f1': 71.70979572051463}

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/squad-test/checkpoint-7050"
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = """
🤗 트랜스포머는 가장 인기 있는 세 가지 딥 러닝 라이브러리인 Jax, PyTorch, TensorFlow를 지원하며, 이들 라이브러리 간의 원활한 통합을 통해
통합되어 있습니다. 한 라이브러리로 모델을 학습시킨 후 다른 라이브러리로 추론할 수 있도록 로드하는 것이 간단합니다.
"""
question = "🤗 트랜스포머가 지원하는 딥 러닝 라이브러리는 무엇인가요?"
question_answerer(question=question, context=context)

{'score': 0.7344798445701599, 'start': 36, 'end': 39, 'answer': 'Jax'}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()
trainer.push_to_hub("commit_message="Training complete")

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator(fp16=True)
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-finetuned-squad-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

In [ ]:
output_dir = "bert-finetuned-squad-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )
    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

In [ ]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)